In [1]:
#import seaborn as sns
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from tiny.tfidf import *
from tiny.usage import *

 
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from xgboost import XGBClassifier

from tiny.lda import *
from  tiny.util import *


model = get_dict()

/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
2018-10-01 08:48:10,469 textcleaner.py[37] INFO 'pattern' package not found; tag filters are not available for English
2018-10-01 08:48:10,476 utils.py[415] INFO loading Word2Vec object from ./output/word2vec.model
2018-10-01 08:48:10,478 smart_open_lib.py[176] DEBUG {'kw': {}, 'mode': 'rb', 'uri': './output/word2vec.model'}
2018-10-01 08:48:10,848 utils.py[449] INFO loading wv recursively from ./output/word2vec.model.wv.* with mmap=None
2018-10-01 08:48:10,850 utils.py[484] INFO setting ignored attribute vectors_norm to None
2018-10-01 08:48:

In [5]:

def get_data():

    model = get_dict(False)
    df = pd.DataFrame(index=list(model.wv.vocab.keys()), data=model[model.wv.vocab])
    df.index.name = 'package'

    df = df.reset_index()

    df['check'] = df.package.apply(lambda val: len(val))

    df = extend_pkg_label(df)
    df.sort_values('check', ascending=False)
    df = convert_label_encode(df, ['p_type'])

    train = df[df.p_type != 'Unknown']
    test = df[df.p_type == 'Unknown']
    return train, test

train, test = get_data()
X = train.iloc[:, 1:21]
y = train['p_sub_type']
X

2018-10-01 08:49:13,481 utils.py[415] INFO loading Word2Vec object from ./output/word2vec.model
2018-10-01 08:49:13,485 smart_open_lib.py[176] DEBUG {'kw': {}, 'mode': 'rb', 'uri': './output/word2vec.model'}
2018-10-01 08:49:13,857 utils.py[449] INFO loading wv recursively from ./output/word2vec.model.wv.* with mmap=None
2018-10-01 08:49:13,859 utils.py[484] INFO setting ignored attribute vectors_norm to None
2018-10-01 08:49:13,861 utils.py[449] INFO loading vocabulary recursively from ./output/word2vec.model.vocabulary.* with mmap=None
2018-10-01 08:49:13,864 utils.py[449] INFO loading trainables recursively from ./output/word2vec.model.trainables.* with mmap=None
2018-10-01 08:49:13,866 utils.py[484] INFO setting ignored attribute cum_table to None
2018-10-01 08:49:13,868 utils.py[421] INFO loaded ./output/word2vec.model
/Users/lali2/dev/python/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed 

['package', 'p_sub_type', 'combine_type'] will convert to label encode, and fillna with Other


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.650840,0.601984,0.131018,-0.731730,-0.178072,0.868449,-1.322020,-0.148938,-1.111573,0.012307,0.188634,-1.103032,0.440752,1.442625,-1.632704,-0.996701,-0.625133,-1.355493,1.480869,0.295517
4,-1.424341,1.120771,1.739148,-3.691791,-1.680478,0.485942,-0.735106,2.193806,0.102495,-0.313782,3.884208,0.519560,1.509282,0.142046,-1.109845,0.810541,-2.035270,-1.552717,0.927096,0.611965
7,-2.986012,-0.814389,2.781976,-2.745912,-0.666466,0.682631,-1.548478,0.460384,-1.662480,2.007880,2.106773,-1.192236,-0.388477,-1.007308,-1.450636,0.996483,-0.787162,0.204513,2.164048,-0.071988
8,2.956400,1.167142,0.504569,-2.480989,-1.078090,-0.203467,-1.831539,0.426001,-2.625649,-1.367852,0.136612,-2.496227,1.151269,3.424044,-1.959227,1.214161,0.521686,-3.371470,-0.640321,-2.198957
19,2.419349,1.484813,1.027162,0.833978,-6.538847,-4.411415,-2.824739,-1.041580,-2.010087,2.453049,0.717144,0.496865,-0.097826,-2.120788,-0.168943,1.025234,-0.232043,-2.566198,0.301437,-2.878966
22,0.270371,2.603549,-0.166328,3.457898,-0.362062,-1.367385,-1.868712,1.045475,-2.503233,1.728817,-1.133675,3.400445,-1.046970,-0.502601,4.214144,-0.831588,-5.900730,0.951524,-0.031132,-2.502102
23,0.771711,-1.116683,0.868999,0.882189,-2.421249,-0.854654,0.945609,-1.711923,0.381042,-0.481744,-0.547240,1.346747,-2.433124,-1.639089,1.257759,0.181785,-0.504044,0.169598,0.385908,-1.447061
27,1.214032,0.849157,-0.247024,-0.372965,-2.192994,-1.220757,-2.333187,-2.282860,3.051216,-1.398806,-1.052568,3.385062,-1.417546,0.281985,-0.217431,-0.830575,-2.838073,0.216905,2.338010,-0.289848
30,0.052370,-2.551824,4.337920,0.582730,-0.008570,-2.667681,0.668269,1.278108,-0.252293,-1.327406,4.924891,1.569401,0.523632,-1.499904,-1.623848,2.836075,5.059356,-0.725153,0.053338,0.192769
31,-0.920064,2.383770,-1.499236,1.865022,2.123928,1.168265,-2.028928,0.071890,-3.609489,0.919168,-2.266611,-0.010443,-0.565588,-0.043617,0.061825,-0.740408,-0.851148,0.857003,-0.306368,1.266365


In [14]:
print( np.unique(y_train) )
print( np.unique(y_test) )

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [16]:

gbm = XGBClassifier(
                objective='multi:softprob',
                eval_metric='mlogloss',
                num_class=22,
                n_estimators=20,
                max_depth=3,

                min_child_weight=1,
                learning_rate=0.1,

                silent=True,
                gamma=0,
                max_delta_step=0,
                subsample=1,
                colsample_bytree=1,
                colsample_bylevel=1,
                reg_alpha=1,
                reg_lambda=1,
                scale_pos_weight=1,
                seed=1,
                missing=None)
# print(random_search.grid_scores_)
gbm.fit(X_train, y_train,  eval_set=[(X_test, y_test)], early_stopping_rounds=50, verbose=True )

results = gbm.evals_result()

print(results)

[0]	validation_0-mlogloss:3.05177
Will train until validation_0-mlogloss hasn't improved in 50 rounds.
[1]	validation_0-mlogloss:3.01924
[2]	validation_0-mlogloss:2.99001
[3]	validation_0-mlogloss:2.96482
[4]	validation_0-mlogloss:2.94357
[5]	validation_0-mlogloss:2.92285
[6]	validation_0-mlogloss:2.90434
[7]	validation_0-mlogloss:2.88765
[8]	validation_0-mlogloss:2.87236
[9]	validation_0-mlogloss:2.8587
[10]	validation_0-mlogloss:2.84676
[11]	validation_0-mlogloss:2.8353
[12]	validation_0-mlogloss:2.82478
[13]	validation_0-mlogloss:2.81524
[14]	validation_0-mlogloss:2.80597
[15]	validation_0-mlogloss:2.79737
[16]	validation_0-mlogloss:2.79012
[17]	validation_0-mlogloss:2.78346
[18]	validation_0-mlogloss:2.77655
[19]	validation_0-mlogloss:2.77088
{'validation_0': {'mlogloss': [3.051766, 3.019241, 2.990009, 2.964824, 2.943568, 2.922853, 2.904342, 2.887646, 2.872357, 2.858696, 2.846756, 2.835304, 2.824777, 2.815245, 2.805973, 2.797369, 2.790115, 2.783456, 2.776547, 2.770876]}}


[3.051766,
 3.019241,
 2.990009,
 2.964824,
 2.943568,
 2.922853,
 2.904342,
 2.887646,
 2.872357,
 2.858696,
 2.846756,
 2.835304,
 2.824777,
 2.815245,
 2.805973,
 2.797369,
 2.790115,
 2.783456,
 2.776547,
 2.770876]

In [19]:
best_epoch = np.array(results['validation_0']['mlogloss']).argmin()+1
best_score = np.array(results['validation_0']['mlogloss']).min()

print(f'{best_epoch}_{best_score}')


20_2.770876
